In [1]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("/kaggle/input/kazsandra-kz/full.csv")

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
print(train_df.columns)
print("Train samples:", len(train_df))
print(train_df.head())

texts = train_df['text'].astype(str).tolist()

print("Example text:", texts[0])

Index(['custom_id', 'text', 'label', 'domain'], dtype='object')
Train samples: 126044
           custom_id                                               text  \
115469  pla070479pla                              Бұл ойын маған ұнады☺   
44415   map003666map                    Халык коп !!! Оте таза мешіт!!!   
31070   kas024473kas  Коляска ұнады, көңілімізден шықты, түсі де әдемі.   
94250   pla049260pla  Құрметті разроботчиктер мемлекеттік қызметкеле...   
145794  pla100804pla                             Базар жок амбал болдык   

        label    domain  
115469      1  appstore  
44415       5   mapping  
31070       5    market  
94250       4  appstore  
145794      5  appstore  
Example text: Бұл ойын маған ұнады☺


In [2]:
import pandas as pd
import re

df = train_df[['text', 'label']].rename(columns={'text': 'text'})

def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"[^a-zа-яәіқңғүұһёө\s.,!?-]", "", text, flags=re.IGNORECASE)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['clean_text'] = df['text'].apply(clean_text)

print(df.head())

print("Total processed texts:", len(df))

                                                     text  label  \
115469                              Бұл ойын маған ұнады☺      1   
44415                     Халык коп !!! Оте таза мешіт!!!      5   
31070   Коляска ұнады, көңілімізден шықты, түсі де әдемі.      5   
94250   Құрметті разроботчиктер мемлекеттік қызметкеле...      4   
145794                             Базар жок амбал болдык      5   

                                               clean_text  
115469                               бұл ойын маған ұнады  
44415                     халык коп !!! оте таза мешіт!!!  
31070   коляска ұнады, көңілімізден шықты, түсі де әдемі.  
94250   құрметті разроботчиктер мемлекеттік қызметкеле...  
145794                             базар жок амбал болдык  
Total processed texts: 126044


In [3]:
def map_sentiment(score):
    if score < 2.0:
        return "negative"
    elif score < 4.0:
        return "neutral"
    else:
        return "positive"

df['sentiment'] = df['label'].apply(map_sentiment)

print(df.head())

print(df['sentiment'].value_counts())

                                                     text  label  \
115469                              Бұл ойын маған ұнады☺      1   
44415                     Халык коп !!! Оте таза мешіт!!!      5   
31070   Коляска ұнады, көңілімізден шықты, түсі де әдемі.      5   
94250   Құрметті разроботчиктер мемлекеттік қызметкеле...      4   
145794                             Базар жок амбал болдык      5   

                                               clean_text sentiment  
115469                               бұл ойын маған ұнады  negative  
44415                     халык коп !!! оте таза мешіт!!!  positive  
31070   коляска ұнады, көңілімізден шықты, түсі де әдемі.  positive  
94250   құрметті разроботчиктер мемлекеттік қызметкеле...  positive  
145794                             базар жок амбал болдык  positive  
sentiment
positive    99854
negative    17710
neutral      8480
Name: count, dtype: int64


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    truncation=True,
    max_length=128
)

texts = df['clean_text'].tolist()

results = sentiment_pipeline(texts, batch_size=32)

df['model_sentiment'] = [r['label'].lower() for r in results]

print(df.head())
print(df['model_sentiment'].value_counts())

2025-12-03 11:21:11.332781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764760871.705747      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764760871.827786      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
import pandas as pd

assert 'model_sentiment' in df.columns, "Run Block 5 first."

model_counts = df['model_sentiment'].value_counts()
model_percentages = round((model_counts / len(df)) * 100, 2)

model_summary = pd.DataFrame({
    "count": model_counts,
    "percentage": model_percentages
}).sort_index()

print("=== Model Sentiment Distribution ===")
print(model_summary)
print("\nTotal samples:", len(df))


In [ ]:

df2 = train_df[['text', 'label']].rename(columns={'text': 'text'})
def map_sentiment(score):
    if score < 2.0:
        return "negative"
    elif score < 4.0:
        return "neutral"
    else:
        return "positive"

df2['sentiment'] = df2['label'].apply(map_sentiment)

print(df2.head())

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

assert 'sentiment' in df.columns, "Run Block 3 first."
assert 'model_sentiment' in df.columns, "Run Block 5 first."

labels = ["negative", "neutral", "positive"]
cm = confusion_matrix(df['sentiment'], df['model_sentiment'], labels=labels)

cm_df = pd.DataFrame(cm, index=[f"true_{l}" for l in labels],
                        columns=[f"pred_{l}" for l in labels])

print("=== Confusion Matrix ===")
print(cm_df)

print("\n=== Agreement Report ===")
print(classification_report(df['sentiment'], df['model_sentiment'], labels=labels))

accuracy = (df['sentiment'] == df['model_sentiment']).mean() * 100
print(f"\nOverall agreement: {accuracy:.2f}%")
